# Modelling with fbprophet

This notebook has codes used to build a predictive model with fbprophet using combinations of hyperparameters. However, results were unsuccessful as it took more than 3 days to run and my cores have overheated. 

These are kept here for future considerations when more processing power is available. 

In [ ]:
#prep historical dataframe for prophet
historical_train = train[['Croatia','time']]
historical_train.rename(columns={'time':'ds',
                                'Croatia':'y'}, inplace=True)

In [ ]:
#set param grid 
fb_params = {'changepoint_prior_scale' : np.logspace(0.001,0.3,5),
            'seasonality_prior_scale': np.logspace(0.01,5,5),
            'changepoint_range': [0.3,0.5,0.95],
            'seasonality_mode':['multiplicative','additive']}

In [ ]:
#generate all combinations of parameters
all_params = [dict(zip(fb_params.keys(),v)) for v in itertools.product(*fb_params.values())]
# store results in each of these
rmses = []
maes = []

In [ ]:
%%time
#cross validation to evaluate all params
for param in all_params:
    prophet = Prophet(**param).fit(historical_train) #fit model with all params
    cv_historical_train = cross_validation(prophet,horizon='31 days',parallel='processes')
    perf_historical_train = performance_metrics(cv_historical_train, rolling_window=1)
    rmses.append(perf_historical_train['rmse'].values[0])
    maes.append(perf_historical_train['mae'].values[0])

In [ ]:
#find the best params
results = pd.DataFrame(all_params)
results['rmse'] = rmses
results['mae'] = maes
results['mape'] = mapes

Codes were learnt from fb diagnostics documentation. 
https://facebook.github.io/prophet/docs/diagnostics.html